# A universal framework for fusing semantic information and temporal consistency for background segmentations

This notebook illustrate the evaluation process of the propose model, one can reproduce the results given in the paper. In addition, more details regarding the evaluation process are given here.

---by Zhi Zeng Sep,28,2018

# 1. Temporal consistency based Model

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from matplotlib import pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import os
import time

## 1.0. Check paths

Check paths for the dataset

In [ ]:
dataset_root_path = './Ours_Dataset'
print('dataset_root_path is: \n' + dataset_root_path)

Get the FH model folder for each scene

In [ ]:
model_root_path = './Our_Models'
per_scene_FH_model_root_path = model_root_path+'\\FH\\per_scene'
print('model_root_path is: \n' + model_root_path + '\n')
print('per_scene_FH_model_root_path is: \n' + per_scene_FH_model_root_path)

## 1.1. Load the temporal consistency based model

In [ ]:
from Utilities.fuzzy_partition_histogram import FuzzyHistogramModel

## 1.2. Check the fuzzy histogram based background model

In [ ]:
FH_MODEL = FuzzyHistogramModel()

In [ ]:
with FH_MODEL.graph.as_default():
    for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES ):
        print(var.name,var.shape)
    print(FH_MODEL.indexing_constant)
    for op in tf.get_default_graph().get_operations():
        if op.type == "Placeholder":
            print(op.get_attr)

# 2. DeepLab Model

In [ ]:
import os
from io import BytesIO
import tarfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import tensorflow as tf

## 2.1. Load the model

In [ ]:
from Utilities.deeplab import DeepLabModel,vis_segmentation,vis_segmentation_map_calculate

In [ ]:
model_root_path = './Our_Models'
deeplab_model_path = model_root_path + '\\SS_model\\deeplab_model.tar.gz'
print('deeplab_model_path: \n',deeplab_model_path)

In [ ]:
SS_MODEL = DeepLabModel(deeplab_model_path)
print('model loaded successfully!')

In [ ]:
def run_visualization_local(img_path):
    """Inferences DeepLab model and visualizes result."""
    try:
        orignal_im = Image.open(img_path)
    except IOError:
        print('Cannot retrieve image. Please check path: ' + img_path)
        return

    print('running deeplab on image %s...' % img_path)
    resized_im, seg_logits, seg_map = SS_MODEL.run(orignal_im)
    print('seg_logits.shape: ',seg_logits.shape)
    print('seg_map.shape: ',seg_map.shape)
    
    vis_segmentation(resized_im, seg_map)

img_path = 'in000534.jpg'
run_visualization_local(img_path)
resized_im, seg_logits, seg_map = SS_MODEL.run(Image.open(img_path))

# 3. Top model

In [ ]:
from Utilities.synthesis_model_white_box_rectified_F_score import SynthesisModel,calculate_double_mask,single_feature_builder

### 3.1. Load the model

In [ ]:
TOP_Model = SynthesisModel()

In [ ]:
with TOP_Model.graph.as_default():
    for var in tf.trainable_variables():
        print(var)

## 3.2. Run for each scene

In [ ]:
care_catagory_list = ['baseline',
                      'dynamicBackground',
                      'intermittentObjectMotion',
                      'badWeather',
                      'shadow',
                      'cameraJitter',
                      'lowFramerate']

In [ ]:
per_scene_FH_model_root_path = model_root_path+'\\FH\\per_scene'
trained_TOP_model_path = './Our_Models\\FH\\model'

In [ ]:
for root, _, files in os.walk(dataset_root_path):
    
    relative_path_hierarch_list = root.split('\\')[len(dataset_root_path.split('\\')):]
    level_above_dataset_root_path = len(relative_path_hierarch_list)
    
    truth_file_list = []
    image_file_list = []
    result_file_list = []
    FH_result_file_list = []
    SS_result_file_list = []
    
    if len(relative_path_hierarch_list) == 2 and relative_path_hierarch_list[0] in care_catagory_list:
        
        print(relative_path_hierarch_list[1])
        
        Test_GTs_path = root + '\\Test_GTs'
        Test_Inputs_path = root + '\\Test_Input'
        Test_Results_path = root + '\\FH_Results'
        
        FH_model_path = per_scene_FH_model_root_path + '\\'+relative_path_hierarch_list[0] + '\\' + relative_path_hierarch_list[1] + '\\classic_model'
        FH_MODEL.load_model(model_path=FH_model_path)
        TOP_Model.load_model(model_path=trained_TOP_model_path)
        
        print('Models loaded.')
        
        # Get the testing data
        for _, _, files in os.walk(Test_GTs_path):
            for file in files:
                truth_file_list.append(Test_GTs_path+'\\'+file)
                image_file_list.append(Test_Inputs_path+'\\in'+file[2:8]+'.jpg')
                result_file_list.append(Test_Results_path+'\\result'+file[2:8]+'.png')
                FH_result_file_list.append(Test_Results_path+'\\FH_result'+file[2:8]+'.png')
                SS_result_file_list.append(Test_Results_path+'\\SS_result'+file[2:8]+'.png')
        
        
        for image_file,result_file, FH_result_file, SS_result_file in zip(image_file_list,result_file_list,FH_result_file_list,SS_result_file_list):
            # Read the image
            cv_BGR_image = cv2.resize(cv2.imread(image_file),(320, 240), interpolation = cv2.INTER_CUBIC)
            cv_gray_image = cv2.cvtColor(cv_BGR_image, cv2.COLOR_BGR2GRAY)
            cv_float_gray_image = cv_gray_image.astype('float')/256.0 # Avoid reaching the 22th bin of a histogram (not using ./255)
            cv2.imshow('cv_float_gray_image',cv_float_gray_image)
            cv2.waitKey(1)
            # FH segmentation
            raw_segmentation = FH_MODEL.histogram_checking(cv_float_gray_image)
            cv2.imshow('raw_segmentation',raw_segmentation)
            cv2.waitKey(1)
            # Calculate SS result
            Image_image = Image.open(image_file)
            _, seg_logits, seg_map = SS_MODEL.run(Image_image)
            seg_logits_channel_sum = np.sum(seg_logits,axis=-1)
            seg_logits_channel_sum_tile = np.dstack([seg_logits_channel_sum for i in range(seg_logits.shape[-1])])
            seg_logits_normalized = seg_logits/seg_logits_channel_sum_tile
            seg_map_show = vis_segmentation_map_calculate(seg_map)
            cv2.imshow('seg_map_show',seg_map_show)
            cv2.waitKey(1)
            # Synthesis
            composite_feature = np.expand_dims(single_feature_builder(raw_segmentation,seg_logits_normalized),axis=0)
            synthesis_result = np.squeeze(TOP_Model.estimate(composite_feature)).clip(0.0,1.0)
            synthesis_result_C = cv2.applyColorMap(np.uint8(synthesis_result*255), cv2.COLORMAP_RAINBOW)
            cv2.imshow('synthesis_result',synthesis_result_C)
            composite_feature_2 = np.expand_dims(single_feature_builder(raw_segmentation*0+0.5,seg_logits_normalized),axis=0)
            synthesis_result_2 = np.squeeze(TOP_Model.estimate(composite_feature_2)).clip(0.0,1.0)
            synthesis_result_2_C = cv2.applyColorMap(np.uint8(synthesis_result_2*255), cv2.COLORMAP_RAINBOW)
            cv2.imshow('synthesis_result_2',synthesis_result_2_C)
            composite_feature_3 = np.expand_dims(single_feature_builder(raw_segmentation,seg_logits_normalized*0+1.0/21),axis=0)
            synthesis_result_3 = np.squeeze(TOP_Model.estimate(composite_feature_3)).clip(0.0,1.0)
            synthesis_result_3_C = cv2.applyColorMap(np.uint8(synthesis_result_3*255), cv2.COLORMAP_RAINBOW)
            cv2.imshow('synthesis_result_3',synthesis_result_3_C)
            cv2.waitKey(1)
            # Record results
            cv2.imwrite(result_file,synthesis_result*255)
            cv2.imwrite(SS_result_file,synthesis_result_2*255)
            cv2.imwrite(FH_result_file,synthesis_result_3*255)
            # Update histogram
            FH_MODEL.update_histogram(cv_float_gray_image,synthesis_result,train_flag=False)
            FH_MODEL.reduce_histogram()

cv2.destroyAllWindows()

# 4. Evaluations

## 4.1. Visualize the weights

In [ ]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
import plotly.graph_objs as go
import pandas as pd

init_notebook_mode(connected=True)

weight_index_list = [i for i in range(len(trainable_variable_list))]
weight_name_list = [var.name.split('/')[1]+'_'+var.name.split('/')[1].split(':')[0] for var in trainable_variable_list]
weight_value_list = []
for value in trainable_variable_value_list:
    weight_value_list.append(value)

# make figure
figure = {
    'data': [],
    'layout': {}
}

# fill in most of layout
#figure['layout']['xaxis'] = {'title': 'Output channel number'}
#figure['layout']['yaxis'] = {'title': 'Input channel number'}
    
# make data
observation_data = np.concatenate([weight_value_list[1],weight_value_list[0]],axis=1).T

ss_catagory = [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
]

data_dict = go.Heatmap(z=observation_data,
                       x=ss_catagory,
                       y=['consistent','inconsistent'],
                       colorscale='Viridis')
figure['data'].append(data_dict)

iplot(figure)

## 4.2 Calculate F-scores

In [ ]:
def calculate_scores(result,truth):
    pos_mask,neg_mask = calculate_double_mask(truth)
    pos_mask = np.squeeze(pos_mask)
    neg_mask = np.squeeze(neg_mask)
    
    TP = np.sum(pos_mask*(1.0-result))
    TN = np.sum(neg_mask*result)
    FP = np.sum(neg_mask*(1.0-result))
    FN = np.sum(pos_mask*result)

    Recall = np.maximum(1e-3,TP) / np.maximum(1e-3,TP + FN)
    Specificity = np.maximum(1e-3,TN) / np.maximum(1e-3,TN + FP)
    PWC = 100.0 * np.maximum(1e-3,FN + FP) / np.maximum(1e-3,TP + FN + FP + TN)
    Precision = np.maximum(1e-3,TP) / np.maximum(1e-3,TP + FP)
    F_Measure = (2 * Precision * Recall) / (Precision + Recall)
    
    return TP,TN,FP,FN,Recall,Specificity,PWC,Precision,F_Measure

In [ ]:
bin_result_score_dict = {}
bin_FH_result_score_dict = {}
bin_SS_result_score_dict = {}
for root, _, files in os.walk(dataset_root_path):
    
    relative_path_hierarch_list = root.split('\\')[len(dataset_root_path.split('\\')):]
    level_above_dataset_root_path = len(relative_path_hierarch_list)
    
    truth_file_list = []
    image_file_list = []
    result_file_list = []
    FH_result_file_list = []
    SS_result_file_list = []
    result_score_list = []
    FH_result_score_list = []
    SS_result_score_list = []
    
    if len(relative_path_hierarch_list) == 2 and relative_path_hierarch_list[0] in care_catagory_list:
        
        print(relative_path_hierarch_list[1])
        
        Test_GTs_path = root + '\\Test_GTs'
        Test_Results_path = root + '\\FH_Results'
        Test_Inputs_path = root + '\\Test_Input'
        
        # Get the files
        for _, _, files in os.walk(Test_GTs_path):
            for file in files:
                truth_file_list.append(Test_GTs_path+'\\'+file)
                image_file_list.append(Test_Inputs_path+'\\in'+file[2:8]+'.jpg')
                result_file_list.append(Test_Results_path+'\\result'+file[2:8]+'.png')
                FH_result_file_list.append(Test_Results_path+'\\FH_result'+file[2:8]+'.png')
                SS_result_file_list.append(Test_Results_path+'\\SS_result'+file[2:8]+'.png')
        
        
        for image_file,truth_file,result_file,FH_result_file,SS_result_file in zip(image_file_list,
                                                                                   truth_file_list,
                                                                                   result_file_list,
                                                                                   FH_result_file_list,
                                                                                   SS_result_file_list):
            # Read the image
            cv_BGR_image = cv2.resize(cv2.imread(image_file),(320, 240), interpolation = cv2.INTER_CUBIC)
            cv2.imshow('cv_BGR_image',cv_BGR_image)
            cv2.waitKey(1)
            
            # Load the groundtruth
            cv_BGR_truth = cv2.resize(cv2.imread(truth_file),(320, 240), interpolation = cv2.INTER_NEAREST)
            cv_gray_truth = cv2.cvtColor(cv_BGR_truth, cv2.COLOR_BGR2GRAY)
            cv2.imshow('cv_gray_truth',cv_gray_truth)
            cv2.waitKey(1)
                    
            # Load the result_file
            cv_BGR_result = cv2.imread(result_file)
            cv_gray_result = cv2.cvtColor(cv_BGR_result, cv2.COLOR_BGR2GRAY)
            cv_float_gray_result = cv_gray_result.astype('float')/255.0 
            cv_float_bin_result = cv_float_gray_result
            cv_float_bin_result[cv_float_bin_result>=0.5]=1.0
            cv_float_bin_result[cv_float_bin_result<0.5]=0.0
            cv2.imshow('cv_float_bin_result',cv_float_bin_result)
            cv2.waitKey(1)
            
            # Load the FH_result_file
            cv_BGR_FH_result = cv2.imread(FH_result_file)
            cv_gray_FH_result = cv2.cvtColor(cv_BGR_FH_result, cv2.COLOR_BGR2GRAY)
            cv_float_gray_FH_result = cv_gray_FH_result.astype('float')/255.0
            cv_float_bin_FH_result = cv_float_gray_FH_result
            cv_float_bin_FH_result[cv_float_bin_FH_result>=0.5]=1.0
            cv_float_bin_FH_result[cv_float_bin_FH_result<0.5]=0.0
            cv2.imshow('cv_float_bin_FH_result',cv_float_bin_FH_result)
            cv2.waitKey(1)

            # Load the SS_result_file
            cv_BGR_SS_result = cv2.imread(SS_result_file)
            cv_gray_SS_result = cv2.cvtColor(cv_BGR_SS_result, cv2.COLOR_BGR2GRAY)
            cv_float_gray_SS_result = cv_gray_SS_result.astype('float')/255.0 
            cv_float_bin_SS_result = cv_float_gray_SS_result
            cv_float_bin_SS_result[cv_float_bin_SS_result>=0.666666]=1.0
            cv_float_bin_SS_result[cv_float_bin_SS_result<0.666666]=0.0
            cv2.imshow('cv_float_bin_SS_result',cv_float_bin_SS_result)
            cv2.waitKey(1)

            result_score_list.append(np.array(calculate_scores(cv_float_bin_result,cv_gray_truth)))
            FH_result_score_list.append(np.array(calculate_scores(cv_float_bin_FH_result,cv_gray_truth)))
            SS_result_score_list.append(np.array(calculate_scores(cv_float_bin_SS_result,cv_gray_truth)))
        
        # Record scores for the cato.
        bin_result_score_dict[relative_path_hierarch_list[1]] = result_score_list
        bin_FH_result_score_dict[relative_path_hierarch_list[1]] = FH_result_score_list
        bin_SS_result_score_dict[relative_path_hierarch_list[1]] = SS_result_score_list
            
cv2.destroyAllWindows()

In [ ]:
bin_result_with_objects_mean_score_dict = {}
bin_FH_result_with_objects_mean_score_dict = {}
bin_SS_result_with_objects_mean_score_dict = {}
for key in bin_result_score_dict.keys():
    bin_result_with_objects_mean_score_dict[key] = np.mean(np.array(bin_result_score_dict[key])[(np.array(bin_result_score_dict[key])[:,0]+np.array(bin_result_score_dict[key])[:,3])>0,:],axis=0)
for key in bin_FH_result_score_dict.keys():
    bin_FH_result_with_objects_mean_score_dict[key] = np.mean(np.array(bin_FH_result_score_dict[key])[(np.array(bin_FH_result_score_dict[key])[:,0]+np.array(bin_FH_result_score_dict[key])[:,3])>0,:],axis=0)
for key in bin_SS_result_score_dict.keys():
    bin_SS_result_with_objects_mean_score_dict[key] = np.mean(np.array(bin_SS_result_score_dict[key])[(np.array(bin_SS_result_score_dict[key])[:,0]+np.array(bin_SS_result_score_dict[key])[:,3])>0,:],axis=0)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Bar(
    x=[key for key in bin_result_with_objects_mean_score_dict.keys()],
    y=[bin_result_with_objects_mean_score_dict[key][-1] for key in bin_result_with_objects_mean_score_dict.keys()],
    name='F-scores of results using fused information',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace1 = go.Bar(
    x=[key for key in bin_FH_result_with_objects_mean_score_dict.keys()],
    y=[bin_FH_result_with_objects_mean_score_dict[key][-1] for key in bin_FH_result_with_objects_mean_score_dict.keys()],
    name='F-scores of results using only temporal consistency',
    marker=dict(
        color='rgb(245,130,45)'
    )
)

trace2 = go.Bar(
    x=[key for key in bin_SS_result_with_objects_mean_score_dict.keys()],
    y=[bin_SS_result_with_objects_mean_score_dict[key][-1] for key in bin_SS_result_with_objects_mean_score_dict.keys()],
    name='F-scores of results using only semantic information',
    marker=dict(
        color='rgb(45,230,45)'
    )
)

data = [trace0, trace1, trace2]
layout = go.Layout(
    xaxis=dict(tickangle=-45),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Check the average boost for F-score

In [ ]:
score_A = np.array([bin_result_with_objects_mean_score_dict[key][-1] for key in bin_result_with_objects_mean_score_dict.keys()])
score_B = np.array([bin_FH_result_with_objects_mean_score_dict[key][-1] for key in bin_FH_result_with_objects_mean_score_dict.keys()])
score_C = np.array([bin_SS_result_with_objects_mean_score_dict[key][-1] for key in bin_SS_result_with_objects_mean_score_dict.keys()])
print(np.mean((score_A-score_B)/(score_B)))
print(np.mean((score_A-score_C)/(score_C)))

## 4.3 Check the Recall

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Bar(
    x=[key for key in bin_result_with_objects_mean_score_dict.keys()],
    y=[bin_result_with_objects_mean_score_dict[key][4] for key in bin_result_with_objects_mean_score_dict.keys()],
    name='Recall of of results using fused information',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace1 = go.Bar(
    x=[key for key in bin_FH_result_with_objects_mean_score_dict.keys()],
    y=[bin_FH_result_with_objects_mean_score_dict[key][4] for key in bin_FH_result_with_objects_mean_score_dict.keys()],
    name='Recall of results using only temporal consistency',
    marker=dict(
        color='rgb(245,130,45)'
    )
)

trace2 = go.Bar(
    x=[key for key in bin_SS_result_with_objects_mean_score_dict.keys()],
    y=[bin_SS_result_with_objects_mean_score_dict[key][4] for key in bin_SS_result_with_objects_mean_score_dict.keys()],
    name='Recall of results using only semantic information',
    marker=dict(
        color='rgb(45,230,45)'
    )
)

data = [trace0, trace1, trace2]
layout = go.Layout(
    xaxis=dict(tickangle=-45),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Check the average boost

In [ ]:
recall_A = np.array([bin_result_with_objects_mean_score_dict[key][4] for key in bin_result_with_objects_mean_score_dict.keys()])
recall_B = np.array([bin_FH_result_with_objects_mean_score_dict[key][4] for key in bin_FH_result_with_objects_mean_score_dict.keys()])
recall_C = np.array([bin_SS_result_with_objects_mean_score_dict[key][4] for key in bin_SS_result_with_objects_mean_score_dict.keys()])
print(np.mean((recall_A-recall_B)/(recall_B)))
print(np.mean((recall_A-recall_C)/(recall_C)))

## Check the Precision 

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Bar(
    x=[key for key in bin_result_with_objects_mean_score_dict.keys()],
    y=[bin_result_with_objects_mean_score_dict[key][-2] for key in bin_result_with_objects_mean_score_dict.keys()],
    name='Precision of of results using fused information',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace1 = go.Bar(
    x=[key for key in bin_FH_result_with_objects_mean_score_dict.keys()],
    y=[bin_FH_result_with_objects_mean_score_dict[key][-2] for key in bin_FH_result_with_objects_mean_score_dict.keys()],
    name='Precision of results using only temporal consistency',
    marker=dict(
        color='rgb(245,130,45)'
    )
)

trace2 = go.Bar(
    x=[key for key in bin_SS_result_with_objects_mean_score_dict.keys()],
    y=[bin_SS_result_with_objects_mean_score_dict[key][-2] for key in bin_SS_result_with_objects_mean_score_dict.keys()],
    name='Precision of results using only semantic information',
    marker=dict(
        color='rgb(45,230,45)'
    )
)

data = [trace0, trace1, trace2]
layout = go.Layout(
    xaxis=dict(tickangle=-45),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Check the average boost

In [ ]:
prec_A = np.array([bin_result_with_objects_mean_score_dict[key][-2] for key in bin_result_with_objects_mean_score_dict.keys()])
prec_B = np.array([bin_FH_result_with_objects_mean_score_dict[key][-2] for key in bin_FH_result_with_objects_mean_score_dict.keys()])
prec_C = np.array([bin_SS_result_with_objects_mean_score_dict[key][-2] for key in bin_SS_result_with_objects_mean_score_dict.keys()])
print(np.mean((prec_A-prec_B)/(prec_B)))
print(np.mean((prec_A-prec_C)/(prec_C)))